In [ ]:
import numpy as np
import random as rd
import autograd.numpy as np
from matplotlib.pyplot import ion, draw, Rectangle, Line2D
import matplotlib.pyplot as plt
from CartPole import *
from sklearn import linear_model
from scipy import interpolate

# Task 2.2

### Linear Model

In [ ]:
def linear_regression(x, y):
    C = np.zeros([4,4])
    C = np.linalg.lstsq(x,y)
    return C

# replacing model.predict(x_test) 
def predict_Y(C, X):
    y_predict = np.matmul(C,np.transpose(X))

In [ ]:
x = CartPole()
m = 500
labels = ['Cart location', 'Cart Velocity', 'Pole Angle', 'Pole Velocity']
initial_state,step_state = np.zeros([m,4]),np.zeros([m,4])
for i in range(m):
    x.setState([1,1,np.random.uniform(-np.pi,np.pi),1])
    initial_state[i] = x.getState()
    x.performAction(10)
    step_state[i] = x.getState()
    x.reset()
change_in_state = step_state - initial_state
print(step_state)

In [ ]:
def Linear_Reg(x,y):
    C = linear_regression(x,y)
    m=500
    y_pred = np.zeros([m,4])
    for i in range(n):
        y_pred[i] = C @ intial_state[i]
    labels = ['Cart location', 'Cart Velocity', 'Pole Angle', 'Pole Velocity']
    xx= y - x
    y_change = y_pred - x
    for j in range(4):
        for i in range(4):
            #plt.scatter(xx[:,[i]],yy[:,[j]],label= 'Actual ' ,color='red',s=20)
            plt.scatter(x_test[:,i],y_test[:,j],label= 'Actual ' ,color='red',s=20)
            plt.scatter(x_test[:,j],y_test[:,j],label = 'Predicted ',color='blue', s= 12)
            #plt.scatter(y_test[:,[j]],y_pred[:,[j]],label = 'Predicted ',color='blue', s= 12)
            plt.legend()
            plt.xlabel('Input '+labels[i])
            plt.ylabel('Prediction  '+labels[j])
            plt.title('Prediction vs Input '+labels[i])
            plt.show()

### Time Evolution

In [ ]:
x = CartPole()
m = 500
labels = ['Cart location', 'Cart Velocity', 'Pole Angle', 'Pole Velocity']
ini_state,next_state = np.zeros([m,4]),np.zeros([m,4])
#Initial State
x.setState([0,3,np.pi,3])
#Input Force
x.performAction(10)
ini_state[0,:] = [0,3,np.pi,3]
next_state[0,:] = x.getState()
for i in range(m-1):
    ini_state[i] = x.getState()
    x.performAction()
    x.remap_angle()
    next_state[i] = x.getState()
change_in_state = next_state - ini_state

In [ ]:
C = linear_regression(ini_state, change_in_state)
y_pred = np.zeros([m,4])
#Predict Next state
for i in range(m):
    pred_state_c = C @ ini_state[i]
    y_pred = pred_state_c + ini_state[i]
t=np.arange(0,0.2*250,0.2)
#Plot
for i in range(4):
    #Smooth Curve with interpolate
    x_new = np.linspace(0,50,10000)
    sp = interpolate.make_interp_spline(t, next_state[250:,i])
    y_new_1 = sp(x_new)
    sp2= interpolate.make_interp_spline(t, y_pred[:,i])
    y_pred_1 = sp2(x_new)
    plt.plot(x_new,y_new_1,label= 'Actual ' ,color='red')
    plt.plot(x_new,y_pred_1,label = 'Predicted ',color='blue')
    plt.legend()
    plt.xlim(0,10)
    plt.ylim(0,10)
    plt.xlabel('time(s)')
    plt.ylabel(labels[i])
    plt.title('Time evolution of '+labels[i])
    plt.show()

# Task 2.2, Non-Linear

In [ ]:
#kernel function to evaluate the state varaibles
def kernel_func(x0,x1,sig):
    delta_x = x0-x1
    delta_x_2 = np.zeros([4,1])
    for i in range(4):
        delta_x_2[i]=(delta_x[i])**2
    delta_x_2[2]=np.sin(0.5*(delta_x[2]))**2
    expo = (-0.5)*float(np.dot(sig, delta_x_2))
    kernel = np.e**expo
    return kernel
#range of input for creating data
def r_range(i, minimum,maximum):
    d = i*(maximum-minimum)
    output = np.full(np.shape(i),minimum)+d
    return output
#generate datapoints using sobol_seq
def generate_x(n):
    x_data = sobol_seq.i4_sobol_generate(4,n)
    d_0=r_range(x_data[:,0],-50,50)
    d_1=r_range(x_data[:,1],-10,10)
    d_2=r_range(x_data[:,2],-(np.pi),np.pi)
    d_3=r_range(x_data[:,3],-15,15)
    output = np.column_stack((d_0,d_1,d_2,d_3))
    return output
def K_matrix(a,b,sig_invert):
    A = np.shape(a)[0]
    B = np.shape(b)[0]
    K = np.zeros(shape=(A,B))
    for i in range(A):
        for j in range(B):
            K[i][j] =kernel_func(a[i],b[j],sig_invert)
    return K

n=500
X = generate_x(n)

In [ ]:
#Change in state 
change_s=CartPole()
Y_change_State = np.zeros([n,4])
for i in range(n):
    change_s.setState(X[i,:4])
    change_s.performAction()
    Y_change_State[i]=change_s.getState()-X[i,:4]
#Now generate subset M
M=320
X_M = generate_x(M)
lamd = 1e-3
#length scale parameters for each state variable
one_over_sigma = [3.08e-10, 2.000e-05, 2.7e+00, 6.5e-02,7.1e-4]
invert_sig = [1/np.sqrt(i) for i in one_over_sigma]
print(invert_sig)
#generate K matrix for datapoints
test_data = np.zeros([n,4])
init_s = np.zeros([n,4])
test_y_data = np.zeros([n,4])
for i in range(n):
    init_s[i] = [np.random.uniform(-50,50),np.random.uniform(-10,10),np.random.uniform(-(np.pi),np.pi),np.random.uniform(-15,15)]
    change_s.setState(init_s[i])
    change_s.performAction()
    test_y_data[i]=change_s.getState()-init_s[i]
Y_test_data = test_y_data

In [ ]:
KNN=K_matrix(X,X,one_over_sigma)
KMN=K_matrix(X_M,X,one_over_sigma)
KNM=K_matrix(X,X_M,one_over_sigma)
KMM=K_matrix(X_M,X_M,one_over_sigma)
#alpha = (KMN@KNM + lamda*KMM)^-1(KMN@Y)
#alpha = (c)^-1*(d)
c = KMN @ KNM +lamd*KMM
d = np.zeros([4,M])
matrix_alphas = np.zeros([4,M])
for i in range(4):
    d[i] = KMN @ Y_change_State[:,i]
    matrix_alphas[i]=np.linalg.lstsq(c,d[i],rcond=-1)[0]
KNM_test = K_matrix(test_data,X_M,one_over_sigma)
Y_pred=KNM_test @ matrix_alphas.transpose()
y_diff = Y_pred-Y_test_data
cc=np.sum(y_diff**2)
y_train_pred = KNM @ matrix_alphas.transpose()
y_train_diff = Y_change_State - y_train_pred
c0 = np.sum(y_train_diff**2)
print(cc)
print(c0)
labels_real = ['Real Change in Cart Location', 'Real Change in Cart Velocity', 'Real change in Pole Angle', 'Real Change in Pole Angular Velocity']
labels_pred = ['Predicted Change in Cart Location','Predicted Change in Cart Velocity','Predicted Change in Pole Angle', 'Predicted Change in Pole Angular Velocity']

def real_pred_one_step(n):
    pred_states = np.zeros([n,5])
    next_states = np.zeros([n,4])
    start_states=np.zeros([n,5])
    new_pred=np.zeros([n,4])
    for i in range(n):
        initial_state = [np.random.uniform(-50,50),np.random.uniform(-10,10),np.random.uniform(-(np.pi),np.pi),np.random.uniform(-15,15),np.random.uniform(-20,20)]
        start_states[i]=initial_state
        change_s.setState(start_states[i,:4])
        change_s.performAction(start_states[i,-1])
        next_states[i]=change_s.getState()
        change_s.remap_angle()
    change_s.setState(initial_state)
    for i in range(n):
        pred_states[i] = start_states[i]
        K1M = np.array([kernel_func(pred_states[i],ss, one_over_sigma) for ss in X_M])
        delta_c = K1M @ matrix_alphas.transpose()
        new_pred[i] = delta_c + pred_states[i,:4]        
    return(next_states, pred_states)

def plot_time_evolution(Y_real, Y_pred, n):
    
    cond = f"cart location: {round(Y_real[0][0],2)}, cart velocity: {round(Y_real[0][1],2)}, pole angle: {round(Y_real[0][2],2)}, pole angular velocity: {round(Y_real[0][3],2)}"
    fig1, axs1 = plt.subplots(4)
    fig1.suptitle(f'Real and predicted state variation with time\n {cond}')
    time = np.arange(0, 0.2*n, 0.2)
    states = ["location", "velocity", "pole angle", "pole velocity"]
    index = 0
    for i in range(4):
        labels =['Cart Location', 'Cart Velocity', 'Pole Angle', 'Pole Velocity']
        x_new = np.linspace(0,100,10000)
        sp = interpolate.make_interp_spline(time, Y_real[:,i])
        y_new_1 = sp(x_new)
        sp2= interpolate.make_interp_spline(time, Y_pred[:,i])
        y_pred_1 = sp2(x_new)
        axs1[i].plot(x_new, y_new_1, "black", label ="real")
        axs1[i].plot(x_new, y_pred_1, label = "accurate")
        axs1[i].set(xlabel="Time")
        axs1[i].set(ylabel=states[index])
        axs1[i].set_xlim([0,10])
        index += 1
    axs1[0].legend(bbox_to_anchor=(1.0, 1.1), loc='upper right')
    plt.show()

real_states, pred_states = real_pred_one_step(100)
plot_time_evolution(real_states,pred_states,100)
t = np.arange(0,0.2*100,0.2)


In [ ]:
x = CartPole()
m = 500
t= np.arange(0,0.2*m,0.2)
labels = ['Cart location', 'Cart Velocity', 'Pole Angle', 'Pole Velocity']
ini_state,next_state = np.zeros([m,4]),np.zeros([m,4])
x.setState([0,3,np.pi,3])
x.performAction(10)

ini_state[0,:] = [0,1,np.pi,1]
next_state[0,:] = x.getState()
for i in range(m-1):
    ini_state[i] = x.getState()
    x.performAction()
    #x.remap_angle()
    next_state[i] = x.getState()

y=CartPole()
m=500
labels = ['Cart location', 'Cart Velocity', 'Pole Angle', 'Pole Velocity']
initial_state,step_state = np.zeros([m,4]),np.zeros([m,4])
y.setState([0,3,np.pi,3])
y.performAction(10)
initial_state[0,:] = [0,3,np.pi,3]
step_state[0,:] = y.getState()
t=np.arange(0,m*0.2,0.2)
for i in range(m-1):
    initial_state[i]=y.getState()
    K1M = np.array([kernel_func(initial_state[i],ss, one_over_sigma) for ss in X_M])
    delta_c = K1M @ matrix_alphas.transpose()
    step_state[i] = delta_c + initial_state[i]        
    y.setState(step_state[i])
    #y.remap_angle()


for i in range(4):
    x_new = np.linspace(0,20,10000)
    sp = interpolate.make_interp_spline(t, ini_state[:,i])
    y_new_4 = sp(x_new)
    sp2= interpolate.make_interp_spline(t, initial_state[:,i])
    y_pred_4 = sp2(x_new)

    plt.plot(x_new,y_new_4, '--', color= 'black', label = 'Actual')
    plt.plot(x_new,y_pred_4,'--',color='red', label = 'Predicted')
    plt.xlim(0,5)
    plt.ylim(-10,10)
    plt.xlabel('Time (s)')
    plt.ylabel(labels[i])
    plt.title('Time evolution of State Variable With Force input')
    plt.legend(loc='lower right')
    plt.show()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=180073fb-9c10-447b-b721-e7b1d25fe36a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>